In [1]:
import sys
# adding llms to the system path, make sure to configure the file azureopenai in llms folder with end point details
sys.path.append('C:\\proddev\\research\\aiagentssamples\\llms\\')
import azureopenai
import autogen
from autogen import GroupChat, GroupChatManager
from autogen import AssistantAgent
from autogen import UserProxyAgent


llm_config = {
    "seed": 44,  # for caching the response
    "config_list": azureopenai.config_list,
    "temperature": 0  # for coding it is suggested to be less creative
}

research_assistant = autogen.AssistantAgent(
    name="Researcher",
    llm_config=llm_config,
)

financial_tasks = [
    """What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change?""",
    """Plot a graph for the performance of stocks against ansdaq index."""
]

writing_tasks = ["""Develop an engaging blog post using any information provided."""]

financial_assistant = autogen.AssistantAgent(
    name="Financial_assistant",
    llm_config=llm_config,
)

writer = autogen.AssistantAgent(
    name="writer",
    llm_config=llm_config,
    system_message="""
        You are a professional writer, known for
        your insightful and engaging articles. Generate a html blog post and include charts and tables to make it more engaging.
        You transform complex concepts into compelling narratives.
        Reply "TERMINATE" in the end when everything is done.
        """,
)

user = autogen.UserProxyAgent(
    name="User",
    human_input_mode="ALWAYS",  # ask human for input at each step
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "tasksinput",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

chat_results = user.initiate_chats(
    [
        {
            "recipient": financial_assistant,
            "message": financial_tasks[0],
            "clear_history": True,
            "silent": False,
            "summary_method": "reflection_with_llm",
        },
        {
            "recipient": research_assistant,
            "message": financial_tasks[1],
            "summary_method": "reflection_with_llm",
        },
        {
            "recipient": writer,
            "message": writing_tasks[0],
        },
    ]
)




********************************************************************************
Starting a new chat....

********************************************************************************
User (to Financial_assistant):

What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change?

--------------------------------------------------------------------------------
Financial_assistant (to User):

To get the current stock prices and the performance over the past month for NVDA (NVIDIA) and TSLA (Tesla), we can use a Python script that utilizes a financial data API. One such API is the Yahoo Finance API, which can be accessed using the `yfinance` library in Python.

Here is the plan:
1. Install the `yfinance` library if it's not already installed.
2. Fetch the current stock prices for NVDA and TSLA.
3. Fetch the stock prices from one month ago.
4. Calculate the percentage change over the past month.
5. Print the results.

L